In [91]:
# Load libraries
import csv
import pandas as pd
import numpy as np
import pdb
import copy
import scipy.spatial.distance
import datetime
import gc
from scipy.stats import ttest_ind

In [92]:
# PARAMETERS

# Posible distances: mean_absolute_difference; mean_squared_differences; absolute_difference; squared_differences; cosine; euclidean
distance_metric = 'mean_absolute_difference'

# Assignments to keep (we can use all assignments)
selectedItems = ['Quiz_1_2', 'Quiz_3', 'Quiz_4_5', 'Quiz_6', 'Quiz_7', 'Quiz_8', 'Quiz_9_10', 'Quiz_11', 'Quiz_12', 'Quiz_13', 'Quiz_14', 'Midterm_randxyzHP3X5J4M','Posttest_randxyzX9LZKV0I']

# Mean percentage of items in common
minPercentageCommonItems = 80

# Min items submitted
minPercentageItemsSubmitted = 50

In [93]:
submission_matrix = pd.read_csv("example_submission_matrix.csv", index_col=0)

In [94]:
submission_matrix

,assignment_1,assignment_2,assignment_3,assignment_4,assignment_5
1,2014-06-21T07:53:23.762423+00:00,2014-06-21T08:04:24.962730+00:00,2014-06-21T08:06:12.558066+00:00,2014-06-21T08:04:24.962730+00:00,2014-06-21T08:21:58.590764+00:00
2,2014-05-31T07:59:57.231538+00:00,2014-05-31T08:00:35.057636+00:00,2014-05-31T08:00:35.057636+00:00,2014-05-31T08:00:35.057636+00:00,2014-05-31T08:01:58.247444+00:00
3,2014-05-30T07:50:41.822814+00:00,2014-05-30T07:58:13.736850+00:00,2014-05-30T07:58:13.736850+00:00,2014-05-30T07:55:19.917861+00:00,2014-05-30T16:05:37.965483+00:00
4,NaN,NaN,NaN,NaN,NaN
5,2014-09-05T13:03:14.540994+00:00,NaN,NaN,NaN,2014-09-05T13:05:54.874886+00:00
6,2014-05-30T02:11:59.165730+00:00,2014-05-30T02:13:25.718455+00:00,2014-05-30T02:15:58.231386+00:00,2014-05-30T02:17:31.262157+00:00,2014-05-30T02:23:31.771410+00:00
7,2014-07-25T09:31:45.709167+00:00,2014-07-25T09:31:21.654261+00:00,2014-07-25T09:31:21.654261+00:00,2014-07-25T09:31:21.654261+00:00,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN


In [95]:
# These lines could be used to use only a selection of assignments
# We are using all assignment in the submission matrix now
colsToKeep = []
"""
for col in submission_matrix.columns:
    colsToKeep.append(col in selectedInputFields)

submission_matrix = submission_matrix.iloc[:, colsToKeep]
"""

'\nfor col in submission_matrix.columns:\n    colsToKeep.append(col in selectedInputFields)\n\nsubmission_matrix = submission_matrix.iloc[:, colsToKeep]\n'

In [96]:
percentageItems = round(100*(pd.notna(submission_matrix).sum(axis = 1)/len(submission_matrix.columns)))
# print((percentageItems > minPercentageItems).sum())
# We keep only those accounts that submitted a minimum percentage of items
minItems_submission_matrix = submission_matrix[percentageItems > minPercentageItemsSubmitted]

In [97]:
minItems_submission_matrix

,assignment_1,assignment_2,assignment_3,assignment_4,assignment_5
1,2014-06-21T07:53:23.762423+00:00,2014-06-21T08:04:24.962730+00:00,2014-06-21T08:06:12.558066+00:00,2014-06-21T08:04:24.962730+00:00,2014-06-21T08:21:58.590764+00:00
2,2014-05-31T07:59:57.231538+00:00,2014-05-31T08:00:35.057636+00:00,2014-05-31T08:00:35.057636+00:00,2014-05-31T08:00:35.057636+00:00,2014-05-31T08:01:58.247444+00:00
3,2014-05-30T07:50:41.822814+00:00,2014-05-30T07:58:13.736850+00:00,2014-05-30T07:58:13.736850+00:00,2014-05-30T07:55:19.917861+00:00,2014-05-30T16:05:37.965483+00:00
6,2014-05-30T02:11:59.165730+00:00,2014-05-30T02:13:25.718455+00:00,2014-05-30T02:15:58.231386+00:00,2014-05-30T02:17:31.262157+00:00,2014-05-30T02:23:31.771410+00:00
7,2014-07-25T09:31:45.709167+00:00,2014-07-25T09:31:21.654261+00:00,2014-07-25T09:31:21.654261+00:00,2014-07-25T09:31:21.654261+00:00,NaN
11,2014-06-02T15:47:23.639336+00:00,2014-06-02T15:48:36.508434+00:00,2014-06-02T15:55:32.113046+00:00,2014-06-02T16:02:56.827037+00:00,2014-06-02T16:00:44.603006+00:00
12,2014-06-01T19:57:18.553506+00:00,2014-06-01T19:59:15.103679+00:00,2014-06-01T19:59:15.103679+00:00,2014-06-01T19:59:15.103679+00:00,2014-06-01T20:01:55.874304+00:00
16,2014-05-09T00:27:05.312021+00:00,2014-05-09T00:31:12.552359+00:00,2014-05-09T00:31:44.376429+00:00,2014-05-09T00:31:44.376429+00:00,NaN
17,2014-06-06T02:42:21.433137+00:00,2014-06-06T03:01:48.588642+00:00,2014-06-06T03:01:48.588642+00:00,2014-06-06T03:03:22.699932+00:00,2014-06-06T03:08:57.949438+00:00
19,2014-06-20T12:55:26.343911+00:00,2014-06-22T16:57:18.983302+00:00,2014-06-22T16:58:43.640375+00:00,2014-06-22T16:58:43.640375+00:00,2014-06-22T17:19:59.651668+00:00


In [98]:
# Calculates the distance between two arrays of submissions timestamps:
# arg1: Array of submissions 1
# arg2: Array of submissions 2
# distance_metric: The distance metric we want to compute choice in {absolute_difference, squared_differences, cosine, euclidean}
# return: The distance beteween the two arrays of submissions for the chosen metric
def compute_distance(submissions1, submissions2, distance_metric='absolute_difference', onlyCommonItems=True):
    
    # Here I adapt the two arrays of submissions to take into account only the common items
    if(onlyCommonItems):
        commonMask = (pd.notna(submissions1) & pd.notna(submissions2))      
        percentageCommonItems = 100*(sum(commonMask)/len(submissions1))
        submissions1 = pd.to_datetime(submissions1[commonMask])
        submissions2 = pd.to_datetime(submissions2[commonMask])
        
    if(percentageCommonItems < minPercentageCommonItems):
        return np.nan    
    
    if(distance_metric == 'absolute_difference'):
        distance = np.absolute(submissions1 - submissions2).sum()
    elif(distance_metric == 'mean_absolute_difference'):
        distance = np.absolute(submissions1 - submissions2).sum()/len(submissions1)
    elif(distance_metric == 'squared_differences'):
        distance = pow((submissions1 - submissions2),2).sum()
    elif(distance_metric == 'mean_squared_differences'):
        distance = pow((submissions1 - submissions2),2).sum()/len(submissions1)
    elif(distance_metric == 'euclidean'):
        distance = scipy.spatial.distance.euclidean(submissions1, submissions2)
    elif(distance_metric == 'cosine'):
        distance = scipy.spatial.distance.cosine(submissions1, submissions2)
    
    return round(distance.total_seconds()/60)

In [99]:
# Example of distance computation between two accounts
print(compute_distance(minItems_submission_matrix.iloc[1], minItems_submission_matrix.iloc[4], 'mean_absolute_difference'))

79291


In [ ]:
# Creating distances matrix in minutes
distance_matrix = pd.DataFrame()
print("Starting...")
for index, studentCol in enumerate(minItems_submission_matrix.index):
    if (index % 50) == 0:
        print(index)
    distancesStudentCol = []
    for studentRow in minItems_submission_matrix.index:
        distancesStudentCol.append(compute_distance(minItems_submission_matrix.loc[studentCol], 
                                                    minItems_submission_matrix.loc[studentRow], 
                                                    distance_metric))
    distance_matrix[studentCol] = distancesStudentCol

print("...Finished!")
distance_matrix.index = minItems_submission_matrix.index

Starting...
0
50


In [ ]:
distance_matrix

In [ ]:
# Save distance matrix
# distance_matrix.to_csv("distance_matrix_" + distance_metric + "_" + selectedItems +".csv", sep=";", header=True, na_rep=np.nan, index=False)

In [ ]:
# Load distance matrix
# distance_matrix = pd.read_csv("distance_matrix_" + distance_metric + "_" + selectedItems +".csv", sep=";")

In [ ]:
# Create mask for unique triplets {student1, student2, distance} --
mask = np.ones(distance_matrix.shape, dtype=bool)
mask[np.triu_indices(len(distance_matrix))] = False
#distance_matrix = distance_matrix[mask] --> # THIS WAS THROWING A MEMORY LEAK, IM DOING A SIMPLE LOOP INSTEAD

In [ ]:
counter = 0
studentIDs = list(distance_matrix.columns.values)

students1 = []
students2 = []
distances = []

tripletsDataframe = pd.DataFrame()
print("Starting...")

for index, celBool in np.ndenumerate(mask):
    counter += 1
    if (counter % 200000) == 0:
        print(str((counter/float(distance_matrix.size))*100) + "%")
    if(celBool):
        students1.append(studentIDs[index[0]])
        students2.append(studentIDs[index[1]])
        distances.append(distance_matrix.iloc[index[0],index[1]])

tripletsDataframe['students1'] = students1
tripletsDataframe['students2'] = students2
tripletsDataframe['distances'] = distances

print("...Finished!")

In [ ]:
# Example: Triplets with distances below 30 minutes
tripletsDataframe[tripletsDataframe['distances'] < 30]

In [ ]:
#tripletsDataframe.to_csv("triplets_matrix_" + distance_metric + "_" + selectedItems +".csv", sep=";", header=True, index=False)

In [ ]:
#tripletsDataframe[tripletsDataframe['distances'] < 90].to_csv("selected_triplets_" + distance_metric + "_" + selectedItems +".csv", sep=";", header=True, index=False)